# Extended Events

Some useful Extended Events


## Tracking SQL Logins

Uses John Moorhouse blogpost https://sqlrus.com/2015/06/t-sql-tuesday-67-extended-events/ to monitor SQL Logins

Create the Exteneded Event - Note it drops the session if one named `MonitorSQLLogins` exists

In [ ]:
IF EXISTS(SELECT 1 from sys.server_event_sessions where name = 'MonitorSQLLogins')
    BEGIN
	   DROP EVENT SESSION [MonitorSQLLogins] ON SERVER
    END
GO
CREATE EVENT SESSION [MonitorSQLLogins] ON SERVER 
ADD EVENT sqlserver.connectivity_ring_buffer_recorded(
    ACTION(sqlserver.client_app_name,sqlserver.client_connection_id,sqlserver.client_hostname,sqlserver.context_info,sqlserver.server_principal_name,sqlserver.session_id)
    WHERE ([sqlserver].[session_nt_user]=N'NULL')),
ADD EVENT sqlserver.login(
    ACTION(sqlserver.session_id,sqlserver.session_nt_username,sqlserver.session_server_principal_name)
    WHERE ([sqlserver].[session_nt_user]=N''))
ADD TARGET package0.event_file(SET filename=N'MonitorSQLLogins'),
ADD TARGET package0.histogram(SET filtering_event_name=N'sqlserver.login',source=N'sqlserver.session_server_principal_name')
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=ON,STARTUP_STATE=OFF)
GO

Query the results

In [ ]:
SELECT 
    xed.slot_data.value('(value)[1]', 'varchar(256)') AS 'SQL Login',
    xed.slot_data.value('(@count)[1]', 'varchar(256)') AS NumberOfLogins
FROM (
    SELECT 
        CAST(xet.target_data AS xml)  as target_data
    FROM sys.dm_xe_session_targets AS xet  
    JOIN sys.dm_xe_sessions AS xe  
       ON (xe.address = xet.event_session_address)  
    WHERE xe.name = 'MonitorSQLLogins' 
        and target_name='histogram'
    ) as t
CROSS APPLY t.target_data.nodes('//HistogramTarget/Slot') AS xed (slot_data);
GO

## Tracking Windows Logins

Altered the code from above to also monitor Windows Logins -  - Note it drops the session if one named `MonitorSQLLogins` exists

In [3]:
IF EXISTS(SELECT 1 from sys.server_event_sessions where name = 'MonitorWindowsLogins')
    BEGIN
	   DROP EVENT SESSION [MonitorWindowsLogins] ON SERVER
    END
GO
CREATE EVENT SESSION [MonitorWindowsLogins] ON SERVER 
ADD EVENT sqlserver.connectivity_ring_buffer_recorded(
    ACTION(sqlserver.client_app_name,sqlserver.client_connection_id,sqlserver.client_hostname,sqlserver.context_info,sqlserver.server_principal_name,sqlserver.session_id)
    WHERE ([sqlserver].[session_nt_user]<>N'NULL')),
ADD EVENT sqlserver.login(
    ACTION(sqlserver.session_id,sqlserver.session_nt_username,sqlserver.session_server_principal_name)
    WHERE ([sqlserver].[session_nt_user]<>N''))
ADD TARGET package0.event_file(SET filename=N'MonitorWindowsLogins'),
ADD TARGET package0.histogram(SET filtering_event_name=N'sqlserver.login',source=N'sqlserver.session_server_principal_name')
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=ON,STARTUP_STATE=OFF)
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.115

Query the results

In [ ]:
SELECT 
    xed.slot_data.value('(value)[1]', 'varchar(256)') AS 'SQL Login',
    xed.slot_data.value('(@count)[1]', 'varchar(256)') AS NumberOfLogins
FROM (
    SELECT 
        CAST(xet.target_data AS xml)  as target_data
    FROM sys.dm_xe_session_targets AS xet  
    JOIN sys.dm_xe_sessions AS xe  
       ON (xe.address = xet.event_session_address)  
    WHERE xe.name = 'MonitorWindowsLogins' 
        and target_name='histogram'
    ) as t
CROSS APPLY t.target_data.nodes('//HistogramTarget/Slot') AS xed (slot_data);
GO

## Tracking logins to a particular database

This will dynamically create an extended events session to track logins to a particular database and store in a histogram only - You can remove `Set collect_statement=(0)` and uncomment the `ADD TARGET package0.event_file(SET filename=''' + @XESessionName + '''),` line to capture the statements executed as well but this will get large quickly on busy systems

In [5]:
DECLARE @DatabaseName NVarchar(250) = ''
DECLARE @XESessionName NVarchar(250) = 'MonitorLoginsTo' + @DatabaseName

IF EXISTS(SELECT 1 from sys.server_event_sessions where name = @XESessionName)
    BEGIN
	DECLARE @DROPSqlQuery NVARCHAR(500) = 'DROP EVENT SESSION [' + @XESessionName + '] ON SERVER'
	EXEC (@DROPSqlQuery)   
    END

DECLARE @SqlQuery NVARCHAR(2000) = 'CREATE EVENT SESSION [' +  @XESessionName +  '] ON SERVER 
ADD EVENT sqlserver.sql_statement_completed(SET collect_statement=(0)
    ACTION(sqlserver.client_app_name,sqlserver.client_hostname,sqlserver.username)
      WHERE ([sqlserver].[database_name] = N'''+ @DatabaseName + '''))
-- ADD TARGET package0.event_file(SET filename=''' + @XESessionName + '''),
ADD TARGET package0.histogram(SET filtering_event_name=N''sqlserver.sql_statement_completed'',source=N''sqlserver.username'')
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=OFF,STARTUP_STATE=OFF)
'

EXEC (@SqlQuery)  


Commands completed successfully.

Total execution time: 00:00:00.104

Query Results - Change the Extended event name

In [ ]:
DECLARE @DatabaseName NVarchar(250) = ''
DECLARE @XESessionName NVARCHAR(250) = 'MonitorLoginsTo' + @DatabaseName
SELECT 
    xed.slot_data.value('(value)[1]', 'varchar(256)') AS 'Login',
    xed.slot_data.value('(@count)[1]', 'varchar(256)') AS NumberOfLogins
FROM (
    SELECT 
        CAST(xet.target_data AS xml)  as target_data
    FROM sys.dm_xe_session_targets AS xet  
    JOIN sys.dm_xe_sessions AS xe  
       ON (xe.address = xet.event_session_address)  
    WHERE xe.name = @XESessionName 
        and target_name='histogram'
    ) as t
CROSS APPLY t.target_data.nodes('//HistogramTarget/Slot') AS xed (slot_data);
GO

## Tracking logins using a particular table

This will dynamically create an extended events session to track Ad-Hoc Statements (no sprocs) to a particular table on a  database and store in a histogram only - You can  uncomment the `ADD TARGET package0.event_file(SET filename=''' + @XESessionName + '''),` line to capture the statements executed as well but this will get large quickly on busy systems

In [ ]:
DECLARE @DatabaseName NVarchar(250) = ''
DECLARE @SchemaName NVarchar(250) = ''
DECLARE @TableName NVarchar(250) = ''
DECLARE @XESessionName NVarchar(250) = 'MonitorTableUsage_' + @DatabaseName + '_' + @TableName + '_' + @SchemaName

IF EXISTS(SELECT 1 from sys.server_event_sessions where name = @XESessionName)
    BEGIN
	DECLARE @DROPSqlQuery NVARCHAR(500) = 'DROP EVENT SESSION [' + @XESessionName + '] ON SERVER'
	EXEC (@DROPSqlQuery)   
    END

DECLARE @SqlQuery NVARCHAR(2000) = 'CREATE EVENT SESSION [' +  @XESessionName +  '] ON SERVER
ADD EVENT sqlserver.sql_batch_completed(SET collect_batch_text=(1)
    ACTION(sqlserver.session_server_principal_name)
    WHERE ([sqlserver].[equal_i_sql_unicode_string]([sqlserver].[database_name],N''' + @DatabaseName + ''') AND [sqlserver].[like_i_sql_unicode_string]([batch_text],N''%' + @SchemaName + '%' + @TableName + '%'')))
-- ADD TARGET package0.event_file(SET filename=''' + @XESessionName + '''),
ADD TARGET package0.histogram(SET filtering_event_name=N''sqlserver.sql_batch_completed'',source=N''sqlserver.session_server_principal_name'')
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=OFF,STARTUP_STATE=OFF)
'

EXEC (@SqlQuery)  

Query the data - define all the params in here

In [ ]:
DECLARE @DatabaseName NVarchar(250) = ''
DECLARE @SchemaName NVarchar(250) = ''
DECLARE @TableName NVarchar(250) = ''
DECLARE @XESessionName NVarchar(250) = 'MonitorTableUsage_' + @DatabaseName + '_' + @TableName + '_' + @SchemaName
SELECT 
    xed.slot_data.value('(value)[1]', 'varchar(256)') AS 'Login',
    xed.slot_data.value('(@count)[1]', 'varchar(256)') AS NumberOfLogins
FROM (
    SELECT 
        CAST(xet.target_data AS xml)  as target_data
    FROM sys.dm_xe_session_targets AS xet  
    JOIN sys.dm_xe_sessions AS xe  
       ON (xe.address = xet.event_session_address)  
    WHERE xe.name = @XESessionName 
        and target_name='histogram'
    ) as t
CROSS APPLY t.target_data.nodes('//HistogramTarget/Slot') AS xed (slot_data);
GO